In [42]:
import pandas as pd
import geopandas as gpd
import pydeck as pdk

import hsluv

In [73]:
gdf_hex_gangnam = gpd.read_file('./hexagon_res_10_400m_count/hexagon_with_data.geojson')

gdf_hex_gangnam.head()

,index,x,y,single_archarea,multi_archarea,single_count,multi_count,count_bike_station,count_bus_line,count_subway_line,score_bus,score_bike,score_subway,total_score,geometry
0,8a30e1c8084ffff,127.061695,37.476327,0.0,0.0,0,0,1,8,0,0.975610,0.833333,0.000000,1.808943,"POLYGON ((127.06230 37.47605, 127.06222 37.476..."
1,8a30e1c85aaffff,127.067626,37.493522,0.0,0.0,0,0,2,16,1,1.951220,1.666667,1.666667,5.284553,"POLYGON ((127.06823 37.49324, 127.06815 37.493..."
2,8a30e1ca126ffff,127.059607,37.523075,0.0,0.0,0,0,1,10,0,1.219512,0.833333,0.000000,2.052846,"POLYGON ((127.06021 37.52280, 127.06014 37.523..."
3,8a30e1c88797fff,127.119429,37.463536,0.0,0.0,0,0,0,3,0,0.365854,0.000000,0.000000,0.365854,"POLYGON ((127.12003 37.46326, 127.11996 37.463..."
4,8a30e1ca4257fff,127.022375,37.525698,0.0,0.0,0,0,1,10,0,1.219512,0.833333,0.000000,2.052846,"POLYGON ((127.02298 37.52542, 127.02290 37.526..."


In [25]:
def hex_to_rgb(value):
  value = value.lstrip('#')
  lv = len(value)
  return tuple(int(value[i:i + lv // 3], 16) for i in range(0, lv, lv // 3))

In [49]:
#0 : red
color_start = rgb_to_hsluv([165,0,38])

#5: yello
color_mid = rgb_to_hsluv([254, 224, 139])

#10: green
color_end = rgb_to_hsluv((0, 104, 55))

In [51]:
def interpolate_hsluv(value, min_value, max_value, color_range):
    start_color, end_color = color_range

    # Normalize the value to a range between 0 and 1
    normalized_value = (value - min_value) / (max_value - min_value)

    # Interpolate between the start and end colors in the HSLuv color space
    h = start_color[0] + normalized_value * (end_color[0] - start_color[0])
    s = start_color[1] + normalized_value * (end_color[1] - start_color[1])
    l = start_color[2] + normalized_value * (end_color[2] - start_color[2])

    interpolated_color_hsluv = (h, s, l)

    # Convert the interpolated color from HSLuv to RGB
    r, g, b = hsluv.hsluv_to_rgb(interpolated_color_hsluv)

    return int(r * 255), int(g * 255), int(b * 255)

In [52]:
def colorScaler (score):
  if score >= 10:
    return [0,104,55]
  elif score >= 5:
    return interpolate_hsluv(score, 5, 10, (color_mid, color_end))
  else:
    return interpolate_hsluv(score, 0, 5, (color_start, color_mid))

In [86]:
gdf_hex_gangnam.loc[:,'rgb_color'] = gdf_hex_gangnam.loc[:,'total_score'].apply(colorScaler)

In [87]:
mapbox_api_key = 'pk.eyJ1Ijoic2doYW4iLCJhIjoiY2szamxqbjZnMGtmbTNjbXZzamh4cng3dSJ9.GGv4GVVoZ811d6PKi54PrA';

# Define the data layer
geojson_layer = pdk.Layer(
    'GeoJsonLayer',
    data=gdf_hex_gangnam,
    opacity=0.8,
    stroked=False,
    filled=True,
    extruded=True,
    wireframe=True,
    get_elevation = 0,
    pickable=True,
    get_fill_color='rgb_color',
    get_line_color=[255, 255, 255],
)

In [92]:
tooltip = {
   "html": '''<b>Total Score:</b> {total_score} <br/>
     <b>Bus Score:</b> {score_bus}<br/>
     <b>Subway Score:</b> {score_subway}<br/>
     <b>Bike Score:</b> {score_bike}
     ''',
   "style": {
        "backgroundColor": "steelblue",
        "color": "white",
        "font-family":'sans-serif'
   }
}
view_state = pdk.ViewState(latitude=37.4979, longitude=127.0276, zoom=11)
deck = pdk.Deck(layers=[geojson_layer], initial_view_state=view_state, tooltip = tooltip)

In [93]:
deck.to_html('choropleth_map.html', notebook_display=True)